In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import os
import time
from tqdm import tqdm

In [29]:
csv_file = 'Result_US.csv'
if os.path.exists(csv_file):
    df = pd.read_csv(csv_file)
else:
    data = []
    columns = ['Duration', 'Speaker', 'Title', 'Date', 'Transcript']
    df = pd.DataFrame(data, columns=columns)
df

,Duration,Speaker,Title,Date,Transcript
0,0:00,Alexandra Minna Stern and Natalie Lira,The movement that inspired the Holocaust,Mar 2022,"As a new widow, Sarah Rosas Garcia was already..."
1,21:33,Anna Malaika Tubbs,The critical role of Black mothers — and how t...,Mar 2022,"Anna Malaika Tubbs: ""The Three Mothers"" is abo..."
2,0:00,Kevin Ehrman-Solberg and Kirsten Delegard,The dark history of the suburbs,Mar 2022,"On October 21st, 1909, 125 residents of an af..."
3,0:00,Alex Gendler,Building the impossible: Golden Gate Bridge,Feb 2022,"In the mid-1930s, two familiar spires towered ..."
4,0:00,Sydney Iaukea,The dark history of the overthrow of Hawaii,Feb 2022,"It was January 16th, 1895. Two men arrived at ..."
...,...,...,...,...,...
139,8:38,John Legend,"""Redemption Song""",Jul 2016,"At Free America, we've done a listening and le..."
140,4:02,Peter Paccone,Why is the US Constitution so hard to amend?,May 2016,"When it was ratified in 1789, the U.S. Constit..."
141,13:49,Mary Bassett,Why your doctor should care about social justice,Feb 2016,"When I moved to Harare in 1985, social justice..."
142,16:16,Dalia Mogahed,What it's like to be Muslim in America,Feb 2016,What do you think when you look at me? A woman...


In [30]:
page = 5
topic = 'United+States'
# topic = 'china'
url = "https://www.ted.com/talks?page={}&sort=newest&topics%5B%5D={}".format(page, topic)
print(url)

https://www.ted.com/talks?page=5&sort=newest&topics%5B%5D=United+States


In [31]:
proxies = {
    'http': 'http://127.0.0.1:7890', 'https': 'https://127.0.0.1:7890'
}
res = requests.get(url, proxies)
res.encoding = res.apparent_encoding

In [32]:
soup = BeautifulSoup(res.text, 'lxml')

In [33]:
target = soup.find(id='browse-results')

In [34]:
col_list = target.find_all('div', "col")
print('col_list_num: ' + str(len(col_list)))

col_list_num: 22


In [35]:
# a_list = list(map(lambda x:x.find('a')['href'] ,col_list))
# a_list

In [36]:
tasks_list = tqdm(col_list)
for col in tasks_list:
    # 获取基本信息
    contents = list(col.stripped_strings)
    contents.pop(-2)
    
    # 获取字幕
    t_id = col.find('a')['href'].split('/')[-1]
    tasks_list.set_description('Page {} task {}'.format(page, t_id))
    url = "https://www.ted.com/graphql?operationName=Transcript&variables=%7B%22id%22%3A%22{}%22%2C%22language%22%3A%22en%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%2218f8e983b84c734317ae9388c83a13bc98702921b141c2124b3ce4aeb6c48ef6%22%7D%7D".format(t_id)
    proxies = {
        'http': 'http://127.0.0.1:7890', 'https': 'https://127.0.0.1:7890'
    }
    headers = {
      'Cookie': '_abby=fGNt0UjmN6pIoM5; _abby_hero_form=c; _abby_post15s=b; _abby_web_606_talkpage_membership_conversion=a; _nu=1646903272'
    }
    res2 = requests.request("GET", url, headers=headers, verify=False)
    transcript = ''
    if not ('data' in json.loads(res2.text).keys()):
        print(t_id)
    else:
        data = json.loads(res2.text)['data']
        if data['translation']:
            transcriptsList = []
            for cues in data['translation']['paragraphs']:
                for cue in cues['cues']:
                    text = cue['text']
                    text = text.replace('\n', ' ')
                    transcriptsList.append(text)
            transcript = ' '.join(transcriptsList)
        else:
            pass
        
    # 汇总单个视频的所有信息
    contents.append(transcript)

    # 将信息添加到df里
    df.loc[len(df.index)] = (contents)
    print(t_id + ' done')
    time.sleep(5)

df

Page 5 task yanis_varoufakis_capitalism_will_eat_democracy_unless_we_speak_up:   0%|                                                                               | 0/22 [00:00<?, ?it/s]D:\APP\Anaconda\envs\Scraper\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ted.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


yanis_varoufakis_capitalism_will_eat_democracy_unless_we_speak_up done


Page 5 task david_rothkopf_how_fear_drives_american_politics:   5%|████                                                                                    | 1/22 [00:06<02:23,  6.84s/it]D:\APP\Anaconda\envs\Scraper\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ted.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


david_rothkopf_how_fear_drives_american_politics done


Page 5 task rich_benjamin_my_road_trip_through_the_whitest_towns_in_america:   9%|██████▋                                                                  | 2/22 [00:12<02:06,  6.34s/it]D:\APP\Anaconda\envs\Scraper\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ted.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

david_rockwell_a_memorial_at_ground_zero done


Page 5 task anna_deavere_smith_four_american_characters:  95%|███████████████████████████████████████████████████████████████████████████████████████▊    | 21/22 [02:40<00:06,  6.72s/it]D:\APP\Anaconda\envs\Scraper\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ted.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


anna_deavere_smith_four_american_characters done


Page 5 task anna_deavere_smith_four_american_characters: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [02:47<00:00,  7.59s/it]


,Duration,Speaker,Title,Date,Transcript
0,0:00,Alexandra Minna Stern and Natalie Lira,The movement that inspired the Holocaust,Mar 2022,"As a new widow, Sarah Rosas Garcia was already..."
1,21:33,Anna Malaika Tubbs,The critical role of Black mothers — and how t...,Mar 2022,"Anna Malaika Tubbs: ""The Three Mothers"" is abo..."
2,0:00,Kevin Ehrman-Solberg and Kirsten Delegard,The dark history of the suburbs,Mar 2022,"On October 21st, 1909, 125 residents of an af..."
3,0:00,Alex Gendler,Building the impossible: Golden Gate Bridge,Feb 2022,"In the mid-1930s, two familiar spires towered ..."
4,0:00,Sydney Iaukea,The dark history of the overthrow of Hawaii,Feb 2022,"It was January 16th, 1895. Two men arrived at ..."
...,...,...,...,...,...
161,12:21,Ed Gavagan,A story about knots and surgeons,Sep 2012,"You know, we wake up in the morning, you get d..."
162,4:34,Jay Walker,The world's English mania,May 2009,Let's talk about manias. Let's start with Beat...
163,11:11,Ben Katchor,Comics of bygone New York,Apr 2009,"I'm going to read a few strips. These are, mos..."
164,24:37,David Rockwell,A memorial at Ground Zero,Jun 2007,"Kurt Andersen: Like many architects, David is ..."


In [37]:
os.getcwd()
df.to_csv(csv_file, index=0) #相对位置，保存在getwcd()获得的路径下